In [1]:
#!pip install gspread
#!pip install oauth2client
#!pip install PyGithub
#!pip install google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib
#!pip install openpyxl  # для работы с .xlsx

## Загрузка данных из Google Sheets в GitHub

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import os
import json

In [3]:
# Авторизация
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
with open(r'D:\PycharmProjects\config.ini\credentials.json', 'r', encoding='utf-8') as f:
    creds_data = json.load(f)

creds = ServiceAccountCredentials.from_json_keyfile_dict(creds_data, scope)
client = gspread.authorize(creds)

# Открытие таблицы
sheet = client.open_by_key('1draL0IcMP94h3YChWrHMxfI0AG3gTBgNoXO3lqj0jKE').worksheet('product info')

In [4]:
# Чтение данных
data = sheet.get_all_records()
df = pd.DataFrame(data)
# Сохранение данных в CSV
df.to_csv('products_data.csv', index=False, encoding='utf-8')

In [5]:
from github import Github
from github import GithubException

In [6]:
# Чтение токена из файла
with open(r"D:\PycharmProjects\config.ini\.env.github", 'r', encoding='utf-8') as f:
    github_token = f.read().strip()

In [7]:
# Подключение к GitHub
g = Github(github_token)
user = g.get_user()  # Получаем текущего пользователя

In [8]:
# Название нового репозитория
repo_name = "Tools-for-Ozon-pricing-strategy-optimizing"

try:
    # Пытаемся получить репозиторий (если он уже существует)
    repo = user.get_repo(repo_name)
    print(f"Репозиторий '{repo_name}' уже существует.")
except GithubException as e:
    if e.status == 404:
        # Если репозитория нет — создаём его
        print(f"Репозиторий '{repo_name}' не найден. Создаём новый...")
        repo = user.create_repo(
            repo_name,
            description="Оптимизация ценовой стратегии для Ozon",
            private=False,  # False = публичный репозиторий
            auto_init=False  # Не создавать README при создании
        )
        print(f"Репозиторий успешно создан: {repo.html_url}")
    else:
        raise e  # Если ошибка не 404 — пробрасываем дальше

Репозиторий 'Tools-for-Ozon-pricing-strategy-optimizing' не найден. Создаём новый...
Репозиторий успешно создан: https://github.com/vitaly22041977/Tools-for-Ozon-pricing-strategy-optimizing


In [9]:
# Загрузка файла в репозиторий
file_path = "products_data.csv"
commit_message = "Add product data from Google Sheets"

try:
    # Проверяем, существует ли уже файл
    repo.get_contents(file_path)
    print(f"Файл '{file_path}' уже существует. Используйте update_file для изменения.")
except GithubException as e:
    if e.status == 404:
        # Если файла нет — создаём его
        with open('products_data.csv', 'r', encoding='utf-8') as file:
            content = file.read()
        
        repo.create_file(
            path=file_path,
            message=commit_message,
            content=content,
            branch="main"  # Указываем ветку (по умолчанию — main/master)
        )
        print(f"Файл успешно загружен в репозиторий!")
    else:
        raise e

Файл успешно загружен в репозиторий!
